In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
from sklearn.utils import shuffle
import cv2
from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential 
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History

In [ ]:
train_classes = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
#train_classes  #notebook kept crashing had to remove unnnecessary code

In [ ]:
sample_sub = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
#sample_sub  #notebook kept crashing had to remove unnnecessary code

In [ ]:
print('Total of {} images'.format(len(train_classes)))
train_classes.shape
print('Total of {} images'.format(len(sample_sub)))
sample_sub.shape

In [ ]:
from itertools import chain

labels_name = list(chain.from_iterable([tags.split(" ") for tags in train_classes['tags'].values]))
labels_names = set(labels_name)
print("There is {} unique labels including {}".format(len(labels_names), labels_names))

In [ ]:
tags = train_classes['tags'].apply(lambda x: x.split(' '))

# Create a list of all occuring, non-unique tags
all_tags = [item for sublist in list(train_classes['tags'].apply(lambda row: row.split(" ")).values) for item in sublist]

all_tags

In [ ]:
tags_count = pd.DataFrame({'tag': all_tags}).groupby('tag').size().reset_index().sort_values(0, ascending=False).reset_index(drop=True)
tags_count.columns = ['tag', 'count']


tags_count

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
label_maps = pd.DataFrame()
label_maps['tags'] = ['agriculture', 'artisinal_mine', 'bare_ground','blooming','blow_down','clear','cloudy','conventional_mine','cultivation','habitation','haze', 'partly_cloudy','primary','road','selective_logging','slash_burn','water']

# converting labels to numerical classes
label_maps['map'] = encoder.fit_transform(label_maps['tags'])
label_maps

In [ ]:
#dict
label_map = {'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}

In [ ]:
# Loading the training images

x, y = [], []

train_label = shuffle(train_classes, random_state=1)
for image_name, tags in tqdm(train_classes.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x.append(cv2.resize(arr, (64, 64)))
    y.append(targets)

# normalizing train image pixels
x = np.array(x, np.float16)/255.0

In [ ]:
# this splits the dataset into train and validation sets.

from sklearn.model_selection import train_test_split

x = np.array(x)
y = np.array(y)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

In [ ]:
import gc
gc.collect()

In [ ]:
# function for fbeta evaluation metric

import numpy as np
from keras import backend as K
from sklearn.metrics import fbeta_score

def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [ ]:
# base model. 


from keras.layers import Conv2D, Dense, LSTM, Flatten, MaxPooling2D, BatchNormalization, Dropout
from keras.optimizers import Adam, RMSprop, SGD

#with tpu_strategy.scope():
model = keras.Sequential()
model.add(Conv2D(64, 5, 2, activation = "relu", input_shape = (64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(17, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = Adam(), metrics = [fbeta])
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 25, batch_size = 128)
#model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 44, batch_size = 64)

In [ ]:
import gc
gc.collect()

In [ ]:
test_loss, test_accuracy = model.evaluate(x_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

In [ ]:
model.evaluate(x_val, y_val)
print("train fscore: ", fbeta_score(y_train, np.round(model.predict(x_train)), 2,average = 'weighted'))
print("val fscore: ", fbeta_score(y_val, np.round(model.predict(x_val)), 2, average = 'weighted'))


In [ ]:
test = sample_sub[0 : 40669]
files = sample_sub[40669 : ]

In [ ]:
test_img = []

for image_name, tags in tqdm(test.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

for image_name, tags in tqdm(files.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

test_img = np.array(test_img, np.float16)/255.0


In [ ]:
yres = []
predictions = model.predict(test_img, batch_size = 64, verbose = 2)
yres.append(predictions)

In [ ]:
sub = np.array(yres[0])
for i in range (1, len(yres)):
    sub += np.array(yres[i])
submitdf = pd.DataFrame(sub, columns = label_map)

In [ ]:
submitdf

In [ ]:
import gc
del x, y, x_train, y_train, x_val, y_val
gc.collect()

In [ ]:
preds = []
for i in tqdm(range(submitdf.shape[0]), miniters=1000):
    a = submitdf.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a= a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

sample_sub['tags'] = preds
sample_sub.to_csv('submit.csv', index=False)

In [ ]:
sample_sub